In [18]:
!pip3 install xgboost lightgbm catboost

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [19]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.covariance import EllipticEnvelope
from snowflake.snowpark import Session
import config
import random
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

import config
import random

# Connect to Snowflake
def get_snowflake_session():
    connection_parameters = {
        "account": config.SNOWFLAKE_ACCOUNT,
        "user": config.SNOWFLAKE_USER,
        "role": config.SNOWFLAKE_ROLE,
        "warehouse": config.SNOWFLAKE_WAREHOUSE,
        "database": config.SNOWFLAKE_DATABASE,
        "schema": config.SNOWFLAKE_SCHEMA,
        "authenticator": config.SNOWFLAKE_AUTHENTICATOR
    }
    session = Session.builder.configs(connection_parameters).create()
    print(f"Connected to Snowflake as {config.SNOWFLAKE_USER}")
    return session

session = get_snowflake_session()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://blinkist-useast_1_virginia.snowflakecomputing.com/console/login?login_name=meri-kris.jaama%40go1.com&browser_mode_redirect_port=64905&proof_key=o03nCqH4InWLmCyWmoxP5Nuja7jjXovtOJ%2FX5ZN5lPc%3D to authenticate...
Connected to Snowflake as meri-kris.jaama@go1.com


In [20]:
# Load data from Snowflake
query = "SELECT * FROM blinkist_dev.dbt_mjaama.exp_proceeds_input"
df = session.sql(query).to_pandas()
df.columns = df.columns.str.lower()

from country_utils import add_signup_country_group
from data_utils import split_data_by_user_type

df, _, _ = split_data_by_user_type(df)

# Add country groups
df = add_signup_country_group(df)

/Users/meri-kris/Documents/GitHub/blinkist-expected-proceeds/country_utils.py:37: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  us_df = temp_df.loc[(temp_df.report_date >= six_months_ago) & (temp_df.signup_country == "US") & (
/Users/meri-kris/Documents/GitHub/blinkist-expected-proceeds/country_utils.py:54: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  row_df = temp_df.loc[(temp_df.report_date >= six_months_ago) & (


In [21]:
# Ensure date column is datetime
df['report_date'] = pd.to_datetime(df['report_date'])

# Define the target for trial conversion
df['trial_conversion'] = (df['eur_proceeds_d8'] > 0).astype(int)
target = 'trial_conversion'

# Define features and target
categorical_features = ['channel_group', 'marketing_network_id', 'target_market',
                        'signup_country_group', 'signup_client_platform', 'plan_tier']
numerical_features = ['eur_marketing_spend', 'impressions', 'clicks', 
                      'is_trial_autorenewal_on_d0', 'started_content', 
                      'n_content_starts', 'finished_content', 'n_content_finishes']
features = categorical_features + numerical_features

In [22]:
# Outlier detection and removal
outlier_detector = EllipticEnvelope(contamination=0.01)
outliers = outlier_detector.fit_predict(df[numerical_features])
df = df[outliers == 1]

/Users/meri-kris/Library/Python/3.9/lib/python/site-packages/sklearn/covariance/_robust_covariance.py:185: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (31.334211400857374 > 0.997165824224097). You may want to try with a higher value of support_fraction (current value: 0.501).
  warnings.warn(
/Users/meri-kris/Library/Python/3.9/lib/python/site-packages/sklearn/covariance/_robust_covariance.py:185: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (32.118567654602366 > 0.623143689412703). You may want to try with a higher value of support_fraction (current value: 0.501).
  warnings.warn(
/Users/meri-kris/Library/Python/3.9/lib/python/site-packages/sklearn/covariance/_robust_covariance.py:185: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (32.896664470872629 > 2.279325095008621). You may want to try with a higher value of support_fraction (cu

In [23]:
# Define the number of sets and the window sizes
num_sets = 5
train_window_days = 180
test_window_days = 2
gap_days = 9

# Sort data by date
df = df.sort_values(by='report_date')

# Get the date range for the past three months
max_date = df['report_date'].max() - pd.Timedelta(days=8)
three_months_ago = max_date - pd.Timedelta(days=90)

# Generate random test start dates
random_dates = pd.date_range(start=three_months_ago, end=max_date - pd.Timedelta(days=test_window_days), freq='D')
random_test_dates = random.sample(list(random_dates), num_sets)

# Generate train and test sets
train_test_sets = []

for test_start_date in random_test_dates:
    test_end_date = test_start_date + pd.Timedelta(days=test_window_days)
    train_end_date = test_start_date - pd.Timedelta(days=gap_days)
    train_start_date = train_end_date - pd.Timedelta(days=train_window_days)
    
    train_df = df[(df['report_date'] >= train_start_date) & (df['report_date'] < train_end_date)]
    test_df = df[(df['report_date'] >= test_start_date) & (df['report_date'] < test_end_date)]
    
    train_test_sets.append((train_df, test_df))

In [24]:
# Preprocessing pipelines
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
    ])

In [25]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Define classification models
models = {
    'RandomForest': RandomForestClassifier(random_state=42),
    'GradientBoosting': GradientBoostingClassifier(random_state=42),
    'XGBoost': XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
    'LightGBM': LGBMClassifier(random_state=42),
    'CatBoost': CatBoostClassifier(random_state=42, verbose=0)
}

# Parameter grids for hyperparameter tuning
param_grids = {
    'RandomForest': {
        'model__n_estimators': [50, 100, 200],
        'model__max_depth': [None, 10, 20],
        'model__min_samples_split': [2, 5, 10]
    },
    'GradientBoosting': {
        'model__n_estimators': [50, 100, 200],
        'model__learning_rate': [0.01, 0.1, 0.2],
        'model__max_depth': [3, 5, 7]
    },
    'XGBoost': {
        'model__n_estimators': [50, 100, 200],
        'model__learning_rate': [0.01, 0.1, 0.2],
        'model__max_depth': [3, 5, 7]
    },
    'LightGBM': {
        'model__n_estimators': [50, 100, 200],
        'model__learning_rate': [0.01, 0.1, 0.2],
        'model__max_depth': [-1, 10, 20]
    },
    'CatBoost': {
        'model__iterations': [100, 200, 300],
        'model__learning_rate': [0.01, 0.1, 0.2],
        'model__depth': [3, 5, 7]
    }
}


In [26]:
import time

# Train and evaluate models on each set
results = []

for idx, (train_df, test_df) in enumerate(train_test_sets):
    print(f"Processing set {idx+1}/{len(train_test_sets)}")
    X_train, y_train = train_df[features], train_df[target]
    X_test, y_test = test_df[features], test_df[target]
    
    for name, model in models.items():
        print(f"  Training model: {name}")
        start_time = time.time()
        pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('model', model)])
        
        if name in param_grids:
            print(f"    Performing grid search for {name}")
            grid_search = GridSearchCV(pipeline, param_grids[name], cv=3, scoring='f1')
            grid_search.fit(X_train, y_train)
            best_model = grid_search.best_estimator_
            print(f"    Best parameters for {name}: {grid_search.best_params_}")
        else:
            best_model = pipeline.fit(X_train, y_train)
        
        y_pred = best_model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        results.append({'Set': idx+1, 'Model': name, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1 Score': f1})
        elapsed_time = time.time() - start_time
        print(f"  Completed {name} in {elapsed_time:.2f} seconds - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")

Processing set 1/5
  Training model: RandomForest
    Performing grid search for RandomForest
    Best parameters for RandomForest: {'model__max_depth': 10, 'model__min_samples_split': 2, 'model__n_estimators': 50}
  Completed RandomForest in 1300.26 seconds - Accuracy: 0.7712418300653595, Precision: 0.6818181818181818, Recall: 0.06880733944954129, F1 Score: 0.125
  Training model: GradientBoosting
    Performing grid search for GradientBoosting
    Best parameters for GradientBoosting: {'model__learning_rate': 0.01, 'model__max_depth': 3, 'model__n_estimators': 50}
  Completed GradientBoosting in 773.85 seconds - Accuracy: 0.7625272331154684, Precision: 0.0, Recall: 0.0, F1 Score: 0.0
  Training model: XGBoost
    Performing grid search for XGBoost


/Users/meri-kris/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/meri-kris/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [11:51:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/meri-kris/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [11:51:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/meri-kris/Library/Python/3.9/lib/python/site-packages/xgboost/core.py:158: UserWarning: [11:51:23] WARNING: /Users/runner/work/xgboost/xgboost/src/le

    Best parameters for XGBoost: {'model__learning_rate': 0.01, 'model__max_depth': 5, 'model__n_estimators': 100}
  Completed XGBoost in 32.13 seconds - Accuracy: 0.7625272331154684, Precision: 0.0, Recall: 0.0, F1 Score: 0.0
  Training model: LightGBM
    Performing grid search for LightGBM
[LightGBM] [Info] Number of positive: 12371, number of negative: 35488
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001358 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 978
[LightGBM] [Info] Number of data points in the train set: 47859, number of used features: 51
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258488 -> initscore=-1.053840
[LightGBM] [Info] Start training from score -1.053840


/Users/meri-kris/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/meri-kris/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 12370, number of negative: 35489
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001287 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 980
[LightGBM] [Info] Number of data points in the train set: 47859, number of used features: 52
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258468 -> initscore=-1.053949
[LightGBM] [Info] Start training from score -1.053949


...

[LightGBM] [Info] Number of positive: 10403, number of negative: 33834
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001362 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 978
[LightGBM] [Info] Number of data points in the train set: 44237, number of used features: 52
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.235165 -> initscore=-1.179372
[LightGBM] [Info] Start training from score -1.179372


/Users/meri-kris/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Number of positive: 15604, number of negative: 50751
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001569 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 982
[LightGBM] [Info] Number of data points in the train set: 66355, number of used features: 52
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.235159 -> initscore=-1.179404
[LightGBM] [Info] Start training from score -1.179404
    Best parameters for LightGBM: {'model__learning_rate': 0.01, 'model__max_depth': -1, 'model__n_estimators': 50}
  Completed LightGBM in 78.03 seconds - Accuracy: 0.8409090909090909, Precision: 0.0, Recall: 0.0, F1 Score: 0.0
  Training model: CatBoost
    Performing grid search for CatBoost


/Users/meri-kris/Library/Python/3.9/lib/python/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/meri-kris/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


    Best parameters for CatBoost: {'model__depth': 3, 'model__iterations': 300, 'model__learning_rate': 0.01}
  Completed CatBoost in 81.41 seconds - Accuracy: 0.8444055944055944, Precision: 0.5714285714285714, Recall: 0.08791208791208792, F1 Score: 0.1523809523809524


In [28]:
# Calculate average performance for each model
average_results = {}

for result in results:
    model_name = result['Model']
    if model_name not in average_results:
        average_results[model_name] = {'Accuracy': [], 'Precision': [], 'Recall': [], 'F1 Score': []}
    average_results[model_name]['Accuracy'].append(result['Accuracy'])
    average_results[model_name]['Precision'].append(result['Precision'])
    average_results[model_name]['Recall'].append(result['Recall'])
    average_results[model_name]['F1 Score'].append(result['F1 Score'])

for model_name, metrics in average_results.items():
    avg_accuracy = np.mean(metrics['Accuracy'])
    avg_precision = np.mean(metrics['Precision'])
    avg_recall = np.mean(metrics['Recall'])
    avg_f1 = np.mean(metrics['F1 Score'])
    average_results[model_name] = {
        'Avg Accuracy': avg_accuracy,
        'Avg Precision': avg_precision,
        'Avg Recall': avg_recall,
        'Avg F1 Score': avg_f1
    }
    print(f"{model_name} - Avg Accuracy: {avg_accuracy}, Avg Precision: {avg_precision}, Avg Recall: {avg_recall}, Avg F1 Score: {avg_f1}")


RandomForest - Avg Accuracy: 0.8268606286756995, Avg Precision: 0.6253030303030302, Avg Recall: 0.09087965438266425, Avg F1 Score: 0.15680780754304818
GradientBoosting - Avg Accuracy: 0.8206789935149056, Avg Precision: 0.0, Avg Recall: 0.0, Avg F1 Score: 0.0
XGBoost - Avg Accuracy: 0.8206789935149056, Avg Precision: 0.0, Avg Recall: 0.0, Avg F1 Score: 0.0
LightGBM - Avg Accuracy: 0.8206789935149056, Avg Precision: 0.0, Avg Recall: 0.0, Avg F1 Score: 0.0
CatBoost - Avg Accuracy: 0.823439495408684, Avg Precision: 0.47578304048892284, Avg Recall: 0.05631250128381813, Avg F1 Score: 0.09951258377689304


In [27]:
# Calculate average performance for each model
average_results = {}

for result in results:
    model_name = result['Model']
    if model_name not in average_results:
        average_results[model_name] = {'MSE': [], 'MAE': []}
    average_results[model_name]['MSE'].append(result['MSE'])
    average_results[model_name]['MAE'].append(result['MAE'])

for model_name, metrics in average_results.items():
    avg_mse = np.mean(metrics['MSE'])
    avg_mae = np.mean(metrics['MAE'])
    average_results[model_name] = {'Avg MSE': avg_mse, 'Avg MAE': avg_mae}
    print(f"{model_name} - Avg MSE: {avg_mse}, Avg MAE: {avg_mae}")

KeyError: 'MSE'

In [ ]:
# Select the best model based on average MSE
best_model_name = min(average_results, key=lambda x: average_results[x]['Avg MSE'])
best_model_metrics = average_results[best_model_name]

print(f"Best model: {best_model_name} with Avg MSE: {best_model_metrics['Avg MSE']} and Avg MAE: {best_model_metrics['Avg MAE']}")

In [ ]:
# Close Snowflake session
session.close()
print("Snowflake session closed")